In [1]:
import requests
import sqlite3
import pandas as pd
import time

In [5]:

# Load CSV file
df = pd.read_csv('/home/romilly/git/active/s2agr/data/ai-papers-from-man-computer-symbiosis.csv')

In [6]:

# Set up the SQLite database
conn = sqlite3.connect('papers_database.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE papers 
             (authors TEXT, title TEXT, year INTEGER, tldr TEXT, pdf_url TEXT, abstract TEXT)''')

In [7]:

# Semantic Scholar API base URL
api_url = 'https://api.semanticscholar.org/graph/v1/paper/search'

# Iterate over rows in the DataFrame
for index, row in df.iterrows():
    # Prepare API request (modify as needed based on the API documentation)
    payload = {'query': row['Title of Paper'], 'fields': 'title,authors,year,abstract,url,tldr'}
    response = requests.get(api_url, params=payload)

    if response.status_code == 200:
        data = response.json()

        # Extract the relevant data (modify based on the actual structure of the response)
        tldr = data.get('tldr', '')
        pdf_url = data.get('pdf_url', '')
        abstract = data.get('abstract', '')

        # Insert into the database
        c.execute("INSERT INTO papers VALUES (?, ?, ?, ?, ?, ?)",
                  (row['Author(s)'], row['Title of Paper'], row['Year of Publication'], tldr, pdf_url, abstract))

        # Commit after each insertion
        conn.commit()

    # Respect the rate limit
    time.sleep(5)  # Adjust based on the API's rate limit

# Close the connection
conn.close()


KeyboardInterrupt: 